In [ ]:
import soundcard as sc
import pyaudio

# get a list of all speakers:
speakers = sc.all_speakers()
print(speakers)
# get the current default speaker on your system:
default_speaker = sc.default_speaker()
print(default_speaker)
# get a list of all microphones:
mics = sc.all_microphones()
print(mics)
# get the current default microphone on your system:
default_mic = sc.default_microphone()
print(default_mic)

# search for a sound card by substring:
sc.get_speaker('Scarlett')
one_mic = sc.get_microphone('Scarlett')

In [ ]:
import pyaudio

p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
print(numdevices)

for i in range(0, numdevices):
    print("Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

p.terminate()

In [ ]:
import pyaudio
import numpy as np
import time
import scipy.signal as sig
from matplotlib import pyplot as plt

#ANC parameters
CHUNK = 2048
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100

#open PyAudio
p = pyaudio.PyAudio()

freq = 392 #frequency of reference
x=np.arange(0,CHUNK,1) #for calculating the reference each iteration

#secondary path arrays
response_frames = np.array([])
control_frames = np.array([])
control_frames = np.append(control_frames, np.zeros(CHUNK))
count = 0

#ANC arrays
frames = np.array([])
response = np.zeros(CHUNK)
responses = np.array([])
output = np.zeros(CHUNK)
outputs = np.array([])
responses = np.append(responses, np.zeros(CHUNK))
outputs = np.append(outputs, np.zeros(CHUNK))

#learning rate of ANN
LR = 0.0002
#create weights array
W = np.zeros((2,1))

#high pass filter
normal_cutoff = 50/(0.5 * RATE)
b, a = sig.butter(6, normal_cutoff, btype='high', analog=False)

#main function
def main():
    
    #open the audio stream
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    output=True,
                    input=True,
                    input_device_index = 1, 
                    output_device_index = 4, 
                    frames_per_buffer=CHUNK,
                    stream_callback=callback)
    
    #callback function is called while this loop waits
    stream.start_stream()
    while stream.is_active():
        time.sleep(30)
        stream.stop_stream()
    stream.close()
    
    #plot the error over time
    numpydata = np.hstack(frames)
    
    #plot the outputs over time
    numpydata1 = np.hstack(responses)
    
    plt.plot(numpydata1, label = 'Anti-Noise')
    plt.plot(numpydata, 'r-', label = 'Error Signal', )
    plt.title('Error Microphone Noise vs. Anti-Noise Output at 528 Hz')
    plt.xlabel('Sample Number')
    plt.ylabel('Sound Amplitude')
    plt.legend()
    plt.show()
    
    p.terminate()

#called everytime there is new audio to record or play
def callback(in_data, frame_count, time_info, flag):
    global response_frames, control_frames, output_sin, frames, outputs, responses, x, count, S
    
    #record the audio through the microphone
    input_audio = in_data
    
    #measure the transfer function for the first 200 frames
    if (count < 50):
        
        response_frames = np.append(response_frames, np.frombuffer(input_audio, dtype=np.float32))
    
        #make the next output
        if (count < 25):
            output_sin = 0.25*np.sin(2*np.pi*freq*(1/RATE)*x)
        else:
            output_sin = 0.5*np.sin(2*np.pi*freq*(1/RATE)*x)
        x = x+CHUNK

        output_audio = output_sin
        control_frames = np.append(control_frames, output_audio)

        count = count + 1
        y_out = output_audio
     
    #calculate the transfer function in this frame
    elif (count == 50):
        
        #don't care about inputs and outputs
        y_out = np.zeros(CHUNK)
        
        #fourier transforms of input and output
        nfft = 16384 #frequency range for fft

        #cross spectral density between the control output and microphone input is the Fourier Transform of their convolution
        [f,sXY] = sig.csd(control_frames, response_frames, fs=RATE, window='hanning', nperseg=nfft, noverlap=nfft/2, nfft=nfft)
        [f,sXX] = sig.welch(control_frames, fs=RATE, window='hanning', nperseg=nfft, noverlap=nfft/2, nfft=nfft)

        #the transfer function is H = Sxy/Sxx
        H = sXY/sXX

        index = (np.abs(f-freq)).argmin() #select the indices of the transfer function closest to the frequency of the noise

        #take the transfer function of the secondary path at the specific frequency
        S = np.array([np.real(H[index]), np.imag(H[index])])
        np.savetxt('Filter.txt', S)
            
        count = count + 1
     
    #delay to start the disturbance signal
    elif (count > 50 and count < 100):
        
        #don't care about inputs and outputs
        y_out = np.zeros(CHUNK)
        count = count + 1
    
    #do the noise cancelling
    else:
        
        #record the error through the microphone
        error = input_audio
        frames = np.append(frames, np.frombuffer(error, np.float32))
        errorData = sig.lfilter(b, a, np.frombuffer(error, dtype=np.float32))

        #new reference signals
        x_reference_sin = np.sin(2*np.pi*freq*(1/RATE)*x)
        x_reference_cos = np.cos(2*np.pi*freq*(1/RATE)*x)
        x = x+CHUNK

        #frequency response - recall that multiplying by i is the same as a 90 degree phase shift to the right (isin -> cos, icos -> -sin)
        filtered_sin = S[0]*x_reference_sin + S[1]*x_reference_cos  # y = (H_R + iH_I)(sin(wt)) = H_r sin(wt) + iH_I sin(wt)
        filtered_cos = S[0]*x_reference_cos - S[1]*x_reference_sin  # y = (H_R + iH_I)(cos(wt)) = H_r cos(wt) + iH_I cos(wt)

        #update the weights
        W[0] = W[0] - (LR)*np.sum(errorData*filtered_sin)
        W[1] = W[1] - (LR)*np.sum(errorData*filtered_cos)

        #references of next frame
        x_reference_sin1 = np.sin(2*np.pi*freq*(1/RATE)*x)
        x_reference_cos1 = np.cos(2*np.pi*freq*(1/RATE)*x)

        #output calculated with the weights and the next reference frame
        output = W[0]*x_reference_sin1 + W[1]*x_reference_cos1
        
        #response from transfer function and output
        response = (S[0]*W[0]*x_reference_sin1 + S[1]*W[0]*x_reference_cos1) + (S[0]*W[1]*x_reference_cos1 - S[1]*W[1]*x_reference_sin1)
        responses = np.append(responses, response)

        outputs = np.append(outputs, output.astype(np.float32))
        y_out = output
    
    return (y_out.astype(np.float32).tobytes(), pyaudio.paContinue)

main()

In [ ]:
from scipy.io import wavfile

print(S)
plt.plot(control_frames, 'b-', label = 'Control Signal')
plt.plot(response_frames, 'r-', label = 'Response Signal')
plt.xlim(31000, 32000)
plt.show()


#plot the error over time
numpydata = np.hstack(frames)

#plot the outputs over time
numpydata1 = np.hstack(outputs)
numpydata2 = np.hstack(responses)

plt.xlim(200000, 201000)
plt.ylim(-0.2, 0.2)
plt.plot(numpydata, 'r-', label = 'Error Signal')
plt.plot(numpydata1, 'b-', label = 'Output Signal')
plt.plot(numpydata2, 'g-', label = 'Response to Output Signal')
plt.title('Zoomed in Plot of Error Signal vs. Output and Response')
plt.xlabel('Sample Number')
plt.ylabel('Sound Amplitude')
plt.legend()
plt.show()

wavfile.write('Error Signal.wav', 44100, numpydata.astype(np.float32()))